In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import geopandas as gpd

In [2]:
data_path = "data_collected.csv"

df = pd.read_csv(data_path)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246597444 entries, 0 to 246597443
Data columns (total 9 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   year                         int64  
 1   month                        int64  
 2   day_of_week                  object 
 3   hour_of_day                  float64
 4   origin_latitude              float64
 5   origin_longitude             float64
 6   destination_latitude         float64
 7   destination_longitude        float64
 8   estimated_average_ridership  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 16.5+ GB


In [4]:
df

,year,month,day_of_week,hour_of_day,origin_latitude,origin_longitude,destination_latitude,destination_longitude,estimated_average_ridership
0,2022,1,Wednesday,15.0,40.599300,-73.955929,40.677044,-73.865050,0.3298
1,2022,1,Wednesday,15.0,40.851695,-73.937969,40.824073,-73.893064,0.3070
2,2022,1,Wednesday,15.0,40.816104,-73.896435,40.745494,-73.988691,0.3190
3,2022,1,Wednesday,15.0,40.747141,-73.945032,40.747215,-73.993365,0.2860
4,2022,1,Wednesday,15.0,40.660476,-73.830301,40.824783,-73.944216,0.3877
...,...,...,...,...,...,...,...,...,...
246597439,2025,6,Sunday,12.0,40.669847,-73.950466,40.660397,-73.998091,0.2288
246597440,2025,6,Sunday,12.0,40.581011,-73.974574,40.620769,-73.975264,0.5242
246597441,2025,6,Sunday,12.0,40.604556,-73.998168,40.759145,-73.953260,0.2692
246597442,2025,6,Sunday,12.0,40.693241,-73.990642,40.688089,-73.966839,1.1866


In [5]:
unique_origins = df.drop_duplicates(subset=['origin_latitude', 'origin_longitude'])[['origin_latitude', 'origin_longitude']]
unique_destinations = df.drop_duplicates(subset=['destination_latitude', 'destination_longitude'])[['destination_latitude', 'destination_longitude']]

In [6]:
unique_origins

,origin_latitude,origin_longitude
0,40.599300,-73.955929
1,40.851695,-73.937969
2,40.816104,-73.896435
3,40.747141,-73.945032
4,40.660476,-73.830301
...,...,...
3766,40.898379,-73.854376
4367,40.583209,-73.827559
5030,40.608382,-73.815925
15584929,40.697466,-73.993086


In [7]:
unique_destinations

,destination_latitude,destination_longitude
0,40.677044,-73.865050
1,40.824073,-73.893064
2,40.745494,-73.988691
3,40.747215,-73.993365
4,40.824783,-73.944216
...,...,...
3992,40.588034,-73.813641
4495,40.581011,-73.974574
4646,40.583209,-73.827559
3580616,40.697466,-73.993086


In [8]:
gdf_points_origin = gpd.GeoDataFrame(
    unique_origins,
    geometry=gpd.points_from_xy(unique_origins['origin_longitude'], unique_origins['origin_latitude']),
    crs="EPSG:4326"
)

In [9]:
gdf_points_destination = gpd.GeoDataFrame(
    unique_destinations,
    geometry=gpd.points_from_xy(unique_destinations['destination_longitude'], unique_destinations['destination_latitude']),
    crs="EPSG:4326"
)

In [10]:
# Load the NYC NTA shapefile
gdf_ntas = gpd.read_file('./data/nynta2020_25c/nynta2020.shp')

In [11]:
gdf_ntas_origin = gdf_ntas.to_crs(gdf_points_origin.crs)
gdf_ntas_destination = gdf_ntas.to_crs(gdf_points_destination.crs)

In [12]:
origin_points_with_nta = gpd.sjoin(gdf_points_origin, gdf_ntas_origin, how='left', predicate='within')
destination_points_with_nta = gpd.sjoin(gdf_points_destination, gdf_ntas_destination, how='left', predicate='within')

In [13]:
origin_points_with_nta = origin_points_with_nta.drop(columns=[
    'index_right', 
    'Shape_Leng', 
    'Shape_Area', 
    'BoroCode', 
    'BoroName',
    'CountyFIPS', 
    'NTAType', 
    'CDTA2020', 
    'CDTAName'
])

destination_points_with_nta = destination_points_with_nta.drop(columns=[
    'index_right', 
    'Shape_Leng', 
    'Shape_Area', 
    'BoroCode', 
    'BoroName',
    'CountyFIPS', 
    'NTAType', 
    'CDTA2020', 
    'CDTAName'
])

In [14]:
origin_points_with_nta.rename(columns={'NTA2020': 'origin_nta2020', 'NTAName': 'origin_nta_name'}, inplace=True)
destination_points_with_nta.rename(columns={'NTA2020': 'destination_nta2020', 'NTAName': 'destination_nta_name'}, inplace=True)

In [15]:
origin_points_with_nta

,origin_latitude,origin_longitude,geometry,origin_nta2020,origin_nta_name,NTAAbbrev
0,40.599300,-73.955929,POINT (-73.95593 40.5993),BK1502,Madison,Mdsn
1,40.851695,-73.937969,POINT (-73.93797 40.8517),MN1202,Washington Heights (North),WshHts_N
2,40.816104,-73.896435,POINT (-73.89644 40.8161),BX0202,Longwood,Lngwd
3,40.747141,-73.945032,POINT (-73.94503 40.74714),QN0201,Long Island City-Hunters Point,LIC
4,40.660476,-73.830301,POINT (-73.8303 40.66048),QN8381,John F. Kennedy International Airport,JFK
...,...,...,...,...,...,...
3766,40.898379,-73.854376,POINT (-73.85438 40.89838),BX1203,Wakefield-Woodlawn,Wkfld_Wdln
4367,40.583209,-73.827559,POINT (-73.82756 40.58321),QN1403,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,BrzyPt
5030,40.608382,-73.815925,POINT (-73.81592 40.60838),QN1403,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,BrzyPt
15584929,40.697466,-73.993086,POINT (-73.99309 40.69747),BK0201,Brooklyn Heights,BkHts


In [16]:
df = df.merge(
    origin_points_with_nta[['origin_latitude', 'origin_longitude', 'origin_nta2020', 'origin_nta_name']],
    on=['origin_latitude', 'origin_longitude'],
    how='left'
)

In [17]:
df = df.merge(
    destination_points_with_nta[['destination_latitude', 'destination_longitude', 'destination_nta2020', 'destination_nta_name']],
    on=['destination_latitude', 'destination_longitude'],
    how='left'
)

In [18]:
df

,year,month,day_of_week,hour_of_day,origin_latitude,origin_longitude,destination_latitude,destination_longitude,estimated_average_ridership,origin_nta2020,origin_nta_name,destination_nta2020,destination_nta_name
0,2022,1,Wednesday,15.0,40.599300,-73.955929,40.677044,-73.865050,0.3298,BK1502,Madison,BK0505,East New York-City Line
1,2022,1,Wednesday,15.0,40.851695,-73.937969,40.824073,-73.893064,0.3070,MN1202,Washington Heights (North),BX0202,Longwood
2,2022,1,Wednesday,15.0,40.816104,-73.896435,40.745494,-73.988691,0.3190,BX0202,Longwood,MN0501,Midtown South-Flatiron-Union Square
3,2022,1,Wednesday,15.0,40.747141,-73.945032,40.747215,-73.993365,0.2860,QN0201,Long Island City-Hunters Point,MN0501,Midtown South-Flatiron-Union Square
4,2022,1,Wednesday,15.0,40.660476,-73.830301,40.824783,-73.944216,0.3877,QN8381,John F. Kennedy International Airport,MN0903,Hamilton Heights-Sugar Hill
...,...,...,...,...,...,...,...,...,...,...,...,...,...
246597439,2025,6,Sunday,12.0,40.669847,-73.950466,40.660397,-73.998091,0.2288,BK0901,Crown Heights (South),BK0702,Sunset Park (West)
246597440,2025,6,Sunday,12.0,40.581011,-73.974574,40.620769,-73.975264,0.5242,BK1302,Coney Island-Sea Gate,BK1204,Mapleton-Midwood (West)
246597441,2025,6,Sunday,12.0,40.604556,-73.998168,40.759145,-73.953260,0.2692,BK1101,Bensonhurst,MN0801,Upper East Side-Lenox Hill-Roosevelt Island
246597442,2025,6,Sunday,12.0,40.693241,-73.990642,40.688089,-73.966839,1.1866,BK0202,Downtown Brooklyn-DUMBO-Boerum Hill,BK0204,Clinton Hill


In [26]:
keys = [
    "year",
    "month",
    "day_of_week",
    "hour_of_day",
    "origin_nta2020",
    "origin_nta_name",
    "destination_nta2020",
    "destination_nta_name"
]

df_agg = (
    df.groupby(keys, as_index=False, dropna=False)["estimated_average_ridership"]
      .sum()
)

In [27]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52506840 entries, 0 to 52506839
Data columns (total 9 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   year                         int64  
 1   month                        int64  
 2   day_of_week                  object 
 3   hour_of_day                  float64
 4   origin_nta2020               object 
 5   origin_nta_name              object 
 6   destination_nta2020          object 
 7   destination_nta_name         object 
 8   estimated_average_ridership  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 3.5+ GB


In [28]:
df_agg

,year,month,day_of_week,hour_of_day,origin_nta2020,origin_nta_name,destination_nta2020,destination_nta_name,estimated_average_ridership
0,2022,1,Friday,0.0,BK0101,Greenpoint,BK0102,Williamsburg,2.4044
1,2022,1,Friday,0.0,BK0101,Greenpoint,BK0103,South Williamsburg,2.4761
2,2022,1,Friday,0.0,BK0101,Greenpoint,BK0104,East Williamsburg,3.5014
3,2022,1,Friday,0.0,BK0101,Greenpoint,BK0202,Downtown Brooklyn-DUMBO-Boerum Hill,4.9075
4,2022,1,Friday,0.0,BK0101,Greenpoint,BK0203,Fort Greene,2.5759
...,...,...,...,...,...,...,...,...,...
52506835,2025,6,Wednesday,23.0,QN8381,John F. Kennedy International Airport,QN1002,Ozone Park,6.6642
52506836,2025,6,Wednesday,23.0,QN8381,John F. Kennedy International Airport,QN1201,Jamaica,1.2685
52506837,2025,6,Wednesday,23.0,QN8381,John F. Kennedy International Airport,QN1401,Far Rockaway-Bayswater,4.9744
52506838,2025,6,Wednesday,23.0,QN8381,John F. Kennedy International Airport,QN1402,Rockaway Beach-Arverne-Edgemere,8.5055


In [29]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52506840 entries, 0 to 52506839
Data columns (total 9 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   year                         int64  
 1   month                        int64  
 2   day_of_week                  object 
 3   hour_of_day                  float64
 4   origin_nta2020               object 
 5   origin_nta_name              object 
 6   destination_nta2020          object 
 7   destination_nta_name         object 
 8   estimated_average_ridership  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 3.5+ GB


In [30]:
df_agg.to_csv('data_agg.csv', index=False)
